In [4]:
from datetime import datetime, date, timedelta
import calendar
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
from statistics import mode
import plotly.express as px
import plotly.graph_objects as go
import time

import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import plotly.express as px
import plotly.io as pio

import boto3
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [5]:
cities = ['Mont Saint Michel', 'St Malo', "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [6]:
# Utilisation de l'API Open Street Map pour récupérer les coordonnées de villes

url = "https://nominatim.openstreetmap.org/search?"
user_agent = "project_kayak/1.0 (mouton.cl@gmail.com)"

def find_gps(address, format='json'):
    params = {"q": address, "format": format}
    headers = {"User-Agent": user_agent}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Erreur: Réponse avec le code de statut {response.status_code} pour l'adresse '{address}'")
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Erreur: Impossible de parser la réponse JSON pour l'adresse '{address}'")
        print(f"Contenu de la réponse: {response.text}")
        return None

columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(cities):
    response = find_gps(address = city)
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    time.sleep(1)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [7]:
display(df_cities.head())
df_cities.to_csv('files/cities_latlong.csv', index=False)

,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


# Weather

In [8]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Récupérer la clé API
api_key = os.getenv('OPENWEATHERMAP_API_KEY')

In [9]:
# Extraire les coordonnées de la première ville
lat = df_cities.iloc[0]['latitude']
lon = df_cities.iloc[0]['longitude']

# Requête API pour la première ville
response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&appid={api_key}")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en format JSON
    # Maintenant, vous pouvez traiter les données comme vous le souhaitez
    print(data)
else:
    print("La requête a échoué, code d'erreur :", response.status_code)


{'lat': 48.636, 'lon': -1.5115, 'timezone': 'Europe/Paris', 'timezone_offset': 7200, 'current': {'dt': 1726054396, 'sunrise': 1726033059, 'sunset': 1726079269, 'temp': 15.73, 'feels_like': 15.19, 'pressure': 1015, 'humidity': 70, 'dew_point': 10.28, 'uvi': 2.39, 'clouds': 100, 'visibility': 10000, 'wind_speed': 1.79, 'wind_deg': 343, 'wind_gust': 4.92, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]}, 'minutely': [{'dt': 1726054440, 'precipitation': 0}, {'dt': 1726054500, 'precipitation': 0}, {'dt': 1726054560, 'precipitation': 0}, {'dt': 1726054620, 'precipitation': 0}, {'dt': 1726054680, 'precipitation': 0}, {'dt': 1726054740, 'precipitation': 0}, {'dt': 1726054800, 'precipitation': 0}, {'dt': 1726054860, 'precipitation': 0}, {'dt': 1726054920, 'precipitation': 0}, {'dt': 1726054980, 'precipitation': 0}, {'dt': 1726055040, 'precipitation': 0}, {'dt': 1726055100, 'precipitation': 0}, {'dt': 1726055160, 'precipitation': 0}, {'dt': 1726055220,

In [10]:
# Utilisation de l'API Open Weather Map pour récupérer les données météorologiques des 8 derniers jours
def get_weather(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&units=metric&exclude=current,minutely,hourly,alerts&appid={api_key}"
    response = requests.get(url)
    data = response.json()

    if data :
        humidity = data["daily"][0]["humidity"]                     # % d'humidité
        temp_avg = data["daily"][0]["temp"]["day"]                  # Température moyenne
        temp_min = data["daily"][0]["temp"]["min"]                  # Température minimum
        temp_max = data["daily"][0]["temp"]["max"]                  # Température maximum
        temp_perceived = data["daily"][0]["feels_like"]["day"]      # Température ressentie
        wind_speed = data["daily"][0]["wind_speed"]                 # Vitesse du vent en m/s
        clouds = data["daily"][0]["clouds"]                         # % de nuages
        pop = data["daily"][0]["pop"]                               # Probabilité de précipitations, etre 0 et 1
        
        return humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop
    else:
        return None

In [11]:
weather_results = []

for index, row in df_cities.iterrows():
    name = row["name"]
    latitude = row["latitude"]
    longitude = row["longitude"]

    humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop = get_weather(latitude, longitude)

    weather_results.append({"name": name, 
                            "latitude": latitude, 
                            "longitude": longitude, 
                            "Humidity": humidity, 
                            "Temperature_avg": temp_avg,
                            "Temperature_min": temp_min,
                            "Temperature_max": temp_max,
                            "Temperature_perceived": temp_perceived,
                            "Wind_speed": wind_speed,
                            "Cloudiness": clouds,
                            "Precipitation_prob" : round(pop*100) # to have the probability of precipitation in %
                            })

df = pd.DataFrame(weather_results)
df.head()

,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,Mont Saint Michel,48.6359541,-1.511459954959514,70,15.73,13.31,16.48,15.19,8.95,100,100
1,St Malo,48.649518,-2.0260409,58,15.32,14.19,16.86,14.42,10.75,100,100
2,Bayeux,49.2764624,-0.7024738,50,15.77,11.27,16.14,14.71,9.43,100,100
3,Le Havre,49.4938975,0.1079732,57,15.22,14.50,17.02,14.29,12.81,80,100
4,Rouen,49.4404591,1.0939658,60,15.35,10.19,16.09,14.51,8.73,78,100


In [12]:
df = df.reset_index()
df.head()

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,70,15.73,13.31,16.48,15.19,8.95,100,100
1,1,St Malo,48.649518,-2.0260409,58,15.32,14.19,16.86,14.42,10.75,100,100
2,2,Bayeux,49.2764624,-0.7024738,50,15.77,11.27,16.14,14.71,9.43,100,100
3,3,Le Havre,49.4938975,0.1079732,57,15.22,14.50,17.02,14.29,12.81,80,100
4,4,Rouen,49.4404591,1.0939658,60,15.35,10.19,16.09,14.51,8.73,78,100


In [13]:
# Sauvegarder les données en .csv
df.to_csv("files/cities_forecast_weather.csv", index=False)

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer la figure avec des graphiques en barres
fig = make_subplots(rows=2, 
                    cols=3, 
                    start_cell="bottom-left",
                    specs=[[{"type": "xy"}, {"type": "xy"},{"type": "xy"}], 
                           [{"type": "xy"},{"type": "xy"},{"type": "xy"}]],
                    subplot_titles=("Top 10 cities with the least humidity (%)", 
                                    "Top 10 cities with the best day temperature (°C)", 
                                    "Top 10 cities with the least wind speed (m/s)",
                                    "Top 10 cities with the least cloudiness (%)",
                                    "Top 10 cities with the least probability of rain (%)"))

# 1. Top 10 des villes avec le moins d'humidité
fig.add_trace(go.Bar(
    x=df.sort_values(by=['Humidity']).head(10)["name"],
    y=df.sort_values(by=['Humidity']).head(10)["Humidity"],
    text=df.sort_values(by=['Humidity']).head(10)["Humidity"],
    textposition='auto',
    marker_color='blue'
), row=1, col=1)

# 2. Top 10 des villes avec la meilleure moyenne de température
fig.add_trace(go.Bar(
    x=df.sort_values(by=['Temperature_avg'], ascending=False).head(10)["name"],
    y=df.sort_values(by=['Temperature_avg'], ascending=False).head(10)["Temperature_avg"],
    text=df.sort_values(by=['Temperature_avg'], ascending=False).head(10)["Temperature_avg"],
    textposition='auto',
    marker_color='red'
), row=1, col=2)

# 3. Top 10 des villes avec le moins de vent
fig.add_trace(go.Bar(
    x=df.sort_values(by=['Wind_speed']).head(10)["name"],
    y=df.sort_values(by=['Wind_speed']).head(10)["Wind_speed"],
    text=df.sort_values(by=['Wind_speed']).head(10)["Wind_speed"],
    textposition='auto',
    marker_color='green'
), row=1, col=3)

# 4. Top 10 des villes avec le moins de nuages

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Cloudiness"
df['Cloudiness_adjusted'] = df['Cloudiness'] + 0.1

fig.add_trace(go.Bar(
    x=df.sort_values(by=['Cloudiness_adjusted']).head(10)["name"],
    y=df.sort_values(by=['Cloudiness_adjusted']).head(10)["Cloudiness_adjusted"],
    text=df.sort_values(by=['Cloudiness_adjusted']).head(10)["Cloudiness_adjusted"],
    textposition='auto',
    marker_color='purple'
), row=2, col=1)

# 5. Top 10 des villes avec le risque le moins élevé de précipitation

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Precipitation_prob"
df['Precipitation_prob_adjusted'] = df['Precipitation_prob'] + 0.1

fig.add_trace(go.Bar(
    x=df.sort_values(by=['Precipitation_prob_adjusted']).head(10)["name"],
    y=df.sort_values(by=['Precipitation_prob_adjusted']).head(10)["Precipitation_prob_adjusted"],
    text=df.sort_values(by=['Precipitation_prob_adjusted']).head(10)["Precipitation_prob_adjusted"],
    textposition='auto',
    marker_color='orange'
), row=2, col=2)

# Mise à jour du layout
fig.update_layout(
    height=700, 
    width=1400,
    showlegend=False,
    title="Top 10 Cities Based on Weather Conditions",
    xaxis_title="Cities",
    yaxis_title="Value",
    bargap=0.2
)

fig.show()


In [15]:
# Tri par ordre croissant de la catégorie "temperature_avg"
df_cities_sorted = df.sort_values(by='Temperature_avg', ascending=False)

# Affichage des premières lignes du DataFrame trié
df_cities_sorted.head(10)


,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
27,27,Collioure,42.52505,3.0831554,39,25.79,15.59,26.21,25.44,7.27,16,0,16.1,0.1
18,18,Bormes les Mimosas,43.1506968,6.3419285,35,25.14,16.45,25.37,24.62,10.39,7,0,7.1,0.1
20,20,Marseille,43.2961743,5.3699525,47,23.03,18.23,23.84,22.62,13.00,23,0,23.1,0.1
25,25,Aigues Mortes,43.5661521,4.19154,47,22.81,16.76,22.98,22.37,10.36,11,0,11.1,0.1
24,24,Nimes,43.8374249,4.3600687,45,22.63,13.86,22.73,22.12,7.82,33,21,33.1,21.1
22,22,Avignon,43.9492493,4.8059012,48,22.62,15.04,23.18,22.19,9.34,14,87,14.1,87.1
26,26,Saintes Maries de la mer,43.4515922,4.4277202,51,22.61,16.74,23.07,22.26,9.76,52,0,52.1,0.1
19,19,Cassis,43.2140359,5.5396318,45,22.48,17.44,22.96,21.96,12.86,35,0,35.1,0.1
23,23,Uzes,44.0121279,4.4196718,45,22.14,12.18,22.14,21.58,7.09,100,33,100.1,33.1
21,21,Aix en Provence,43.5298424,5.4474738,47,22.00,14.87,22.23,21.48,9.93,24,0,24.1,0.1


In [16]:
top5_city=df_cities_sorted[0:5]
display(top5_city)

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
27,27,Collioure,42.52505,3.0831554,39,25.79,15.59,26.21,25.44,7.27,16,0,16.1,0.1
18,18,Bormes les Mimosas,43.1506968,6.3419285,35,25.14,16.45,25.37,24.62,10.39,7,0,7.1,0.1
20,20,Marseille,43.2961743,5.3699525,47,23.03,18.23,23.84,22.62,13.00,23,0,23.1,0.1
25,25,Aigues Mortes,43.5661521,4.19154,47,22.81,16.76,22.98,22.37,10.36,11,0,11.1,0.1
24,24,Nimes,43.8374249,4.3600687,45,22.63,13.86,22.73,22.12,7.82,33,21,33.1,21.1


In [17]:
# Sauvegarder le dataframe en .csv
top5_city.to_csv("files/top_5city.csv")

In [18]:
#Création du graphique
df_final = df_cities_sorted.astype({"latitude": float, "longitude": float})

fig = px.scatter_mapbox(df_final, lat="latitude", lon="longitude",
                  
                            mapbox_style="carto-positron",zoom=4, width = 800,  height =860,
                            title = 'Moyenne des températures pour les villes du dataset',
                            color='Temperature_avg',
                            size = 'Cloudiness_adjusted'
                          
            )
fig.show()

## Scrapping

In [19]:
# Lancement de la spider contenue dans le dossier Scrap
%run booking_spider.py

2024-09-11 13:33:20 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-09-11 13:33:20 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:42:31) [MSC v.1937 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-09-11 13:33:20 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-11 13:33:20 [py.warnings] WARNING: c:\Users\mouto\anaconda3\envs\jedha\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning:

'2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See 

In [20]:
data_hotels = pd.read_json("files/booking_spider.json")
data_hotels.head()

,city,name,note,url,description,depth,download_timeout,download_slot,download_latency,gps_hotel
0,Nimes,à 10mn des arènes f2 au calme avec garage gratuit,"8,4",https://www.booking.com/hotel/fr/logement-tout...,"Situé à Nîmes, en Occitanie, à proximité du pa...",2,180,www.booking.com,0.596288,"43.82838120,4.36336300"
1,Nimes,Hotel Concorde,"6,9",https://www.booking.com/hotel/fr/concorde-nime...,Situé dans une rue piétonne du centre de Nîmes...,2,180,www.booking.com,0.637275,"43.83612815,4.36126995"
2,Nimes,Studio Nîmes Centre,"8,2",https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,2,180,www.booking.com,0.625533,"43.83976280,4.35694199"
3,Nimes,Appartement avec terrasse panoramique au coeur...,"9,2",https://www.booking.com/hotel/fr/votre-terrass...,"Situé dans le centre de Nîmes, à 14 km du parc...",2,180,www.booking.com,0.639418,"43.83677600,4.35801800"
4,Nimes,ibis budget Nimes Centre Gare,"8,0",https://www.booking.com/hotel/fr/ibis-budget-n...,Situé à seulement 50 mètres de la gare de Nîme...,2,180,www.booking.com,0.735267,"43.83205590,4.36736450"


In [21]:
# Remise en forme des coordonnées des hôtels
data_hotels['coord_lat'] = data_hotels['gps_hotel'].apply(lambda x: x.split(',')[0])
data_hotels['coord_lon'] = data_hotels['gps_hotel'].apply(lambda x: x.split(',')[1])

In [22]:
to_drop = ['gps_hotel', 'depth', 'download_timeout', 'download_slot', 'download_latency']

In [23]:
data_hotels = data_hotels.drop(to_drop, axis=1)
data_hotels.head()

,city,name,note,url,description,coord_lat,coord_lon
0,Nimes,à 10mn des arènes f2 au calme avec garage gratuit,"8,4",https://www.booking.com/hotel/fr/logement-tout...,"Situé à Nîmes, en Occitanie, à proximité du pa...",43.82838120,4.36336300
1,Nimes,Hotel Concorde,"6,9",https://www.booking.com/hotel/fr/concorde-nime...,Situé dans une rue piétonne du centre de Nîmes...,43.83612815,4.36126995
2,Nimes,Studio Nîmes Centre,"8,2",https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,43.83976280,4.35694199
3,Nimes,Appartement avec terrasse panoramique au coeur...,"9,2",https://www.booking.com/hotel/fr/votre-terrass...,"Situé dans le centre de Nîmes, à 14 km du parc...",43.83677600,4.35801800
4,Nimes,ibis budget Nimes Centre Gare,"8,0",https://www.booking.com/hotel/fr/ibis-budget-n...,Situé à seulement 50 mètres de la gare de Nîme...,43.83205590,4.36736450


In [24]:
data_hotels['note'] = data_hotels['note'].apply(lambda x: None if x==0 else x)
data_hotels.head()

,city,name,note,url,description,coord_lat,coord_lon
0,Nimes,à 10mn des arènes f2 au calme avec garage gratuit,"8,4",https://www.booking.com/hotel/fr/logement-tout...,"Situé à Nîmes, en Occitanie, à proximité du pa...",43.82838120,4.36336300
1,Nimes,Hotel Concorde,"6,9",https://www.booking.com/hotel/fr/concorde-nime...,Situé dans une rue piétonne du centre de Nîmes...,43.83612815,4.36126995
2,Nimes,Studio Nîmes Centre,"8,2",https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,43.83976280,4.35694199
3,Nimes,Appartement avec terrasse panoramique au coeur...,"9,2",https://www.booking.com/hotel/fr/votre-terrass...,"Situé dans le centre de Nîmes, à 14 km du parc...",43.83677600,4.35801800
4,Nimes,ibis budget Nimes Centre Gare,"8,0",https://www.booking.com/hotel/fr/ibis-budget-n...,Situé à seulement 50 mètres de la gare de Nîme...,43.83205590,4.36736450


In [25]:
data_hotel_sorted = data_hotels.sort_values(by='note', ascending=False)
hotel_bests = data_hotel_sorted.head(20)
hotel_bests

,city,name,note,url,description,coord_lat,coord_lon
112,Aigues Mortes,Au Cœur des Remparts,"9,9",https://www.booking.com/hotel/fr/au-coeur-des-...,"Situé à Aigues-Mortes, à 24 km du parc des exp...",43.56540100,4.19297300
93,Aigues Mortes,La maison blanche Intra-Muros Free parking,"9,8",https://www.booking.com/hotel/fr/bienvenue-che...,"Doté d'un jardin, l'établissement La maison bl...",43.56452313,4.19255555
10,Nimes,Rougetomette,"9,7",https://www.booking.com/hotel/fr/rougetomette-...,"Situé à Nîmes, le Rougetomette propose une con...",43.83003691,4.36611269
37,Bormes les Mimosas,Cosy studio - Place du Figuier,"9,7",https://www.booking.com/hotel/fr/studio-figuie...,"Set in Bormes-les-Mimosas, 29 km from Chateau ...",43.15145512,6.34296830
28,Bormes les Mimosas,La Recampado,"9,6",https://www.booking.com/hotel/fr/la-recampado-...,"Situé à Bormes-les-Mimosas, à 29 km du château...",43.15097100,6.34211700
38,Bormes les Mimosas,Bel appartement classé 3 étoiles,"9,6",https://www.booking.com/hotel/fr/joli-2-pieces...,"Offrant une vue paisible sur la rue, le Bel ap...",43.14398290,6.33727200
20,Bormes les Mimosas,Studio lumineux - rue du Corsaire,"9,5",https://www.booking.com/hotel/fr/studio-lumine...,"Situated in Bormes-les-Mimosas, 29 km from Cha...",43.15103203,6.34226151
102,Aigues Mortes,Mas de Constance 3pieces standing,"9,5",https://www.booking.com/hotel/fr/mas-de-consta...,"Situé à Aigues-Mortes, le Mas de Constance 3pi...",43.57211540,4.18652590
25,Bormes les Mimosas,Les Terrasses de Lô,"9,5",https://www.booking.com/hotel/fr/les-terrasses...,Doté d'une baignoire en plein air et offrant u...,43.14388500,6.35353600
106,Aigues Mortes,Appartements 3 étoiles terrasse ou patio intra...,"9,5",https://www.booking.com/hotel/fr/appartement-3...,Appartements 3 étoiles terrasse ou patio intra...,43.56535840,4.19275020


In [26]:
data_hotels = data_hotels.replace(r'\n',' ', regex=True)
data_hotels = data_hotels.replace({chr(0x0153): 'oe',
                                   chr(0x2013): '-',
                                   chr(0x2019): '\'',
                                   chr(0x2022): '-'}, regex=True)

data_hotels

,city,name,note,url,description,coord_lat,coord_lon
0,Nimes,à 10mn des arènes f2 au calme avec garage gratuit,"8,4",https://www.booking.com/hotel/fr/logement-tout...,"Situé à Nîmes, en Occitanie, à proximité du pa...",43.82838120,4.36336300
1,Nimes,Hotel Concorde,"6,9",https://www.booking.com/hotel/fr/concorde-nime...,Situé dans une rue piétonne du centre de Nîmes...,43.83612815,4.36126995
2,Nimes,Studio Nîmes Centre,"8,2",https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,43.83976280,4.35694199
3,Nimes,Appartement avec terrasse panoramique au coeur...,"9,2",https://www.booking.com/hotel/fr/votre-terrass...,"Situé dans le centre de Nîmes, à 14 km du parc...",43.83677600,4.35801800
4,Nimes,ibis budget Nimes Centre Gare,"8,0",https://www.booking.com/hotel/fr/ibis-budget-n...,Situé à seulement 50 mètres de la gare de Nîme...,43.83205590,4.36736450
...,...,...,...,...,...,...,...
120,Nimes,Appart'City Collection Nîmes Arènes,"8,7",https://www.booking.com/hotel/fr/appart-39-cit...,None,43.83387353,4.36143396
121,Nimes,Agréable appartement P2 Centre ville au calme.,"8,6",https://www.booking.com/hotel/fr/agreable-appa...,"Offrant une vue sur le jardin, l'Agréable appa...",43.83799940,4.37090630
122,Nimes,B&B HOTEL Nîmes Centre Arènes,"7,7",https://www.booking.com/hotel/fr/b-amp-b-nimes...,None,43.82816140,4.35479020
123,Nimes,Odalys City Nîmes Arènes,"8,6",https://www.booking.com/hotel/fr/odalys-city-n...,"Situé à Nîmes, à 33 km de l'amphithéâtre d'Arl...",43.83406187,4.35899065


In [27]:
def convert_to_float(x):
    try:
        # Si la valeur est un nombre au format texte, on la convertit en float
        return float(str(x).replace(',', '.').replace('"', ''))
    except ValueError:
        # Si la valeur n'est pas convertible en float, on la laisse telle quelle
        return x

# Applique la fonction à la colonne 'description'
data_hotels['description'] = data_hotels['description'].apply(lambda x: convert_to_float(x) if pd.notnull(x) else x)

data_hotels.head()


,city,name,note,url,description,coord_lat,coord_lon
0,Nimes,à 10mn des arènes f2 au calme avec garage gratuit,"8,4",https://www.booking.com/hotel/fr/logement-tout...,"Situé à Nîmes, en Occitanie, à proximité du pa...",43.82838120,4.36336300
1,Nimes,Hotel Concorde,"6,9",https://www.booking.com/hotel/fr/concorde-nime...,Situé dans une rue piétonne du centre de Nîmes...,43.83612815,4.36126995
2,Nimes,Studio Nîmes Centre,"8,2",https://www.booking.com/hotel/fr/studio-boucar...,Le Studio Nîmes Centre propose des hébergement...,43.83976280,4.35694199
3,Nimes,Appartement avec terrasse panoramique au coeur...,"9,2",https://www.booking.com/hotel/fr/votre-terrass...,"Situé dans le centre de Nîmes, à 14 km du parc...",43.83677600,4.35801800
4,Nimes,ibis budget Nimes Centre Gare,"8,0",https://www.booking.com/hotel/fr/ibis-budget-n...,Situé à seulement 50 mètres de la gare de Nîme...,43.83205590,4.36736450


In [28]:
data_hotels.to_csv('files/hotel_infos.csv', encoding='utf-8')

In [29]:
# Inspecter les valeurs uniques dans les colonnes 'coord_lat' et 'coord_lon'
print(hotel_bests['coord_lat'].unique())
print(hotel_bests['coord_lon'].unique())

['43.56540100' '43.56452313' '43.83003691' '43.15145512' '43.15097100'
 '43.14398290' '43.15103203' '43.57211540' '43.14388500' '43.56535840'
 '43.14644500' '43.29076990' '43.29363530' '42.52754300' '43.29218920'
 '43.29774912' '43.30195653' '42.52739229' '43.12282480' '42.52443110']
['4.19297300' '4.19255555' '4.36611269' '6.34296830' '6.34211700'
 '6.33727200' '6.34226151' '4.18652590' '6.35353600' '4.19275020'
 '6.32772400' '5.36721620' '5.37347210' '3.08535500' '5.37001360'
 '5.36706445' '5.37229434' '3.08228340' '6.35686020' '3.09031585']


In [30]:
# Si des chaînes sont concaténées par erreur, on peut les corriger
hotel_bests.loc[:, 'coord_lat'] = hotel_bests['coord_lat'].apply(lambda x: x.split()[0] if isinstance(x, str) and ' ' in x else x)
hotel_bests.loc[:, 'coord_lon'] = hotel_bests['coord_lon'].apply(lambda x: x.split()[0] if isinstance(x, str) and ' ' in x else x)

# Convertir en float pour s'assurer que ce sont bien des nombres
hotel_bests.loc[:, 'coord_lat'] = pd.to_numeric(hotel_bests['coord_lat'], errors='coerce')
hotel_bests.loc[:, 'coord_lon'] = pd.to_numeric(hotel_bests['coord_lon'], errors='coerce')

# Afficher à nouveau la carte après correction
fig_hotels = px.scatter_mapbox(
    hotel_bests,
    lat=hotel_bests["coord_lat"],
    lon=hotel_bests["coord_lon"],
    color="note",
    hover_data={"name": True, "coord_lat": True, "coord_lon": True, "note": True},
    mapbox_style="carto-positron",
    color_continuous_scale=px.colors.sequential.Blues,
    zoom=4,
    width=800
)
fig_hotels.show()


# STOCKAGE AWS S3

In [31]:
# Charger les variables depuis le fichier .env
load_dotenv()

# Récupérer les crédentiels AWS
aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')

bucketname = 'booking-mouton-bucket'

In [32]:
import boto3

# Initialiser la session boto3 avec les crédentiels chargés
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region
)

bucket = s3_client.create_bucket(Bucket=bucketname, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

# Charger les fichiers dans le bucket S3
s3_client.upload_file('files/top_5city.csv', bucketname, 'top_5city.csv')
s3_client.upload_file('files/booking_spider.json', bucketname, 'booking_spider.json')
s3_client.upload_file('files/hotel_infos.csv', bucketname, 'hotel_infos.csv')


BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [ ]:
# Charger les CSV en DataFrame
top_5city_df = pd.read_csv('files/top_5city.csv')
hotel_infos_df = pd.read_csv('files/hotel_infos.csv')

# Charger le fichier JSON en DataFrame
booking_spider_df = pd.read_json('files/booking_spider.json')


In [ ]:
from sqlalchemy import create_engine

# Créer un moteur de base de données (SQLite par exemple)
engine = create_engine('sqlite:///my_database.db')

# Sauvegarder les DataFrames dans des tables SQL
top_5city_df.to_sql('top_5city', con=engine, if_exists='replace', index=False)
hotel_infos_df.to_sql('hotel_infos', con=engine, if_exists='replace', index=False)
booking_spider_df.to_sql('booking_spider', con=engine, if_exists='replace', index=False)


125

In [ ]:
from sqlalchemy import text

# Exemple : lire une table depuis la base SQL avec text()
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM hotel_infos"))
    for row in result:
        print(row)


(0, 'Lille', "Grand' Place of Lille - Superb designer apartment", '7,7', 'https://www.booking.com/hotel/fr/grand-place-of-lille-superb-designer-apartment.fr.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQm4ARfIA ... (132 characters truncated) ... dults=2&no_rooms=1&group_children=0&req_children=0&hpos=21&hapos=21&sr_order=popularity&srpvid=3c7d39ad6ea106a6&srepoch=1725955932&from=searchresults', "Situé à Lille, à proximité de l'Hospice Gantois, du zoo de Lille et de la maison Coilliot, le Grand' Place of Lille - Superb designer apartment dispose d'une connexion Wi-Fi gratuite. ", 50.6349319, 3.0586239)
(1, 'Lille', 'Studio Vieux Lille parking privé', '9,1', 'https://www.booking.com/hotel/fr/studio-vieux-lille-lille.fr.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQm4ARfIAQ_YAQHoAQH4AQOIAgGoAgO ... (110 characters truncated) ... dults=2&no_rooms=1&group_children=0&req_children=0&hpos=23&hapos=23&sr_order=popularity&srpvid=3c7d39ad6ea106a6&srepoch=1725955932&from=

In [ ]:
import sqlite3

# Connexion à la base de données SQLite
conn = sqlite3.connect('my_database.db')

# Chemin du fichier de dump SQL
with open('dump.sql', 'w') as f:
    # Créer un dump de toute la base de données
    for line in conn.iterdump():
        f.write(f'{line}\n')

conn.close()


In [ ]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('my_database.db')

# Exporter uniquement les tables 'top_5city' et 'hotel_infos'
with open('dump_top_5_city_hotel_infos.sql', 'w') as f:
    for line in conn.iterdump():
        if 'CREATE TABLE top_5city' in line or 'CREATE TABLE hotel_infos' in line:
            f.write(f'{line}\n')
        elif 'INSERT INTO "top_5city"' in line or 'INSERT INTO "hotel_infos"' in line:
            f.write(f'{line}\n')

conn.close()
